In [1]:
import pandas as pd
import re
import numpy as np
import datetime as dt
from ftfy import fix_encoding
import collections

In [2]:
dogs = pd.read_csv('dogs.csv' )
travels = pd.read_csv('dogTravel.csv')
colnames=['state' , 'population'] 
states = pd.read_csv('NST-EST2021-POP.csv', names=colnames, header=None)

In [3]:
dogs.columns.values

array(['id', 'org_id', 'url', 'type.x', 'species', 'breed_primary',
       'breed_secondary', 'breed_mixed', 'breed_unknown', 'color_primary',
       'color_secondary', 'color_tertiary', 'age', 'sex', 'size', 'coat',
       'fixed', 'house_trained', 'declawed', 'special_needs',
       'shots_current', 'env_children', 'env_dogs', 'env_cats', 'name',
       'status', 'posted', 'contact_city', 'contact_state', 'contact_zip',
       'contact_country', 'stateQ', 'accessed', 'type.y', 'description',
       'stay_duration', 'stay_cost'], dtype=object)

# 1. Extract all dogs with status that is not adoptable

In [4]:
dogs['status']

0        adoptable
1        adoptable
2        adoptable
3        adoptable
4        adoptable
           ...    
58175    adoptable
58176    adoptable
58177    adoptable
58178    adoptable
58179    adoptable
Name: status, Length: 58180, dtype: object

In [5]:
dogs[dogs['status'] != 'adoptable'][['id','org_id','status', 'posted', 'contact_city', 'contact_state', 'contact_zip',
       'contact_country', 'stateQ', 'accessed']]

,id,org_id,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed
644,41330726,NV173,2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20,NaN
5549,38169117,AZ414,2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20,NaN
10888,45833989,NY98,2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
11983,45515547,NY98,2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
12495,45294115,NY98,2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
12600,45229004,NY1436,2019-07-11T20:34:42+0000,Saugerties,NY,12477,US,CT,2019-09-20,NaN
12613,45227052,NY1436,2019-07-11T14:16:38+0000,Saugerties,NY,12477,US,CT,2019-09-20,NaN
17619,45569380,CA1209,2019-08-10T16:00:35+0000,Bristow,VA,20136,US,DC,2019-09-20,NaN
18611,44694387,MD295,2019-05-14T21:09:27+0000,Silver Spring,MD,20905,US,DC,2019-09-20,NaN
19747,36978896,VA127,2016-12-15T13:33:43+0000,Gettysburg,PA,17325,US,DC,2019-09-20,NaN


In [6]:
col = ['status', 'posted', 'contact_city', 'contact_state', 'contact_zip',
       'contact_country', 'stateQ', 'accessed']
m = dogs['status'] != 'adoptable'

dogs.loc[m,col] = dogs.loc[m , col].shift(axis=1)

In [7]:
dogs[dogs['status'] != 'adoptable'][['id','org_id','status', 'posted', 'contact_city', 'contact_state', 'contact_zip',
       'contact_country', 'stateQ', 'accessed']]

,id,org_id,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed
644,41330726,NV173,NaN,2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20
5549,38169117,AZ414,NaN,2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20
10888,45833989,NY98,NaN,2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20
11983,45515547,NY98,NaN,2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20
12495,45294115,NY98,NaN,2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20
12600,45229004,NY1436,NaN,2019-07-11T20:34:42+0000,Saugerties,NY,12477,US,CT,2019-09-20
12613,45227052,NY1436,NaN,2019-07-11T14:16:38+0000,Saugerties,NY,12477,US,CT,2019-09-20
17619,45569380,CA1209,NaN,2019-08-10T16:00:35+0000,Bristow,VA,20136,US,DC,2019-09-20
18611,44694387,MD295,NaN,2019-05-14T21:09:27+0000,Silver Spring,MD,20905,US,DC,2019-09-20
19747,36978896,VA127,NaN,2016-12-15T13:33:43+0000,Gettysburg,PA,17325,US,DC,2019-09-20


# 2. For each (primary) breed, determine the number of dogs

In [8]:
dogs.groupby('breed_primary').size()

breed_primary
Affenpinscher                         17
Afghan Hound                           4
Airedale Terrier                      19
Akbash                                 3
Akita                                181
                                    ... 
Wirehaired Pointing Griffon            1
Wirehaired Terrier                    60
Xoloitzcuintli / Mexican Hairless     11
Yellow Labrador Retriever            158
Yorkshire Terrier                    360
Length: 216, dtype: int64

# 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

In [9]:
dogs[['breed_primary','breed_secondary']]

,breed_primary,breed_secondary
0,American Staffordshire Terrier,Mixed Breed
1,Pit Bull Terrier,Mixed Breed
2,Shepherd,NaN
3,German Shepherd Dog,NaN
4,Dachshund,NaN
...,...,...
58175,Border Collie,NaN
58176,Australian Shepherd,Australian Cattle Dog / Blue Heeler
58177,Border Collie,NaN
58178,Boxer,Mixed Breed


In [11]:
(dogs[dogs['breed_secondary'].notnull()].groupby('breed_primary').size() / dogs[dogs['breed_secondary'].isnull()].groupby('breed_primary').size())

breed_primary
Affenpinscher                        0.133333
Afghan Hound                         0.333333
Airedale Terrier                     0.900000
Akbash                                    NaN
Akita                                0.403101
                                       ...   
Wirehaired Pointing Griffon               NaN
Wirehaired Terrier                   0.428571
Xoloitzcuintli / Mexican Hairless    0.571429
Yellow Labrador Retriever            0.645833
Yorkshire Terrier                    0.558442
Length: 216, dtype: float64

# 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

In [12]:
dogs['posted'] = pd.to_datetime(dogs['posted'] , format="%Y-%m-%dT%H:%M:%S%z")
dogs['posted']

0       2019-09-20 16:37:59+00:00
1       2019-09-20 16:24:57+00:00
2       2019-09-20 14:10:11+00:00
3       2019-09-20 10:08:22+00:00
4       2019-09-20 06:48:30+00:00
                   ...           
58175   2019-05-03 14:23:49+00:00
58176   2019-04-13 16:20:24+00:00
58177   2018-09-27 04:18:56+00:00
58178   2018-09-12 05:03:38+00:00
58179   2018-09-03 20:42:24+00:00
Name: posted, Length: 58180, dtype: datetime64[ns, UTC]

In [13]:
dogs.groupby('breed_primary')['posted'].min()

breed_primary
Affenpinscher                       2012-03-08 10:27:33+00:00
Afghan Hound                        2017-06-29 23:28:51+00:00
Airedale Terrier                    2014-06-13 12:59:36+00:00
Akbash                              2019-07-21 00:35:59+00:00
Akita                               2012-03-03 09:31:08+00:00
                                               ...           
Wirehaired Pointing Griffon         2016-06-29 20:03:55+00:00
Wirehaired Terrier                  2012-11-27 14:07:54+00:00
Xoloitzcuintli / Mexican Hairless   2007-02-01 00:00:00+00:00
Yellow Labrador Retriever           2010-05-31 00:00:00+00:00
Yorkshire Terrier                   2012-02-04 17:14:52+00:00
Name: posted, Length: 216, dtype: datetime64[ns, UTC]

In [14]:
dogs.groupby('breed_primary')['posted'].max()

breed_primary
Affenpinscher                       2019-09-14 10:10:51+00:00
Afghan Hound                        2019-07-27 00:38:48+00:00
Airedale Terrier                    2019-09-19 18:40:39+00:00
Akbash                              2019-08-23 17:11:04+00:00
Akita                               2019-09-20 15:19:57+00:00
                                               ...           
Wirehaired Pointing Griffon         2016-06-29 20:03:55+00:00
Wirehaired Terrier                  2019-09-19 22:52:45+00:00
Xoloitzcuintli / Mexican Hairless   2019-09-08 11:15:54+00:00
Yellow Labrador Retriever           2019-09-20 06:30:27+00:00
Yorkshire Terrier                   2019-09-20 16:56:38+00:00
Name: posted, Length: 216, dtype: datetime64[ns, UTC]

# 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [15]:
dogs['contact_state'].unique()

array(['NV', 'AZ', 'UT', 'CA', 'AK', 'AL', 'AR', 'CO', 'NY', 'MA', 'CT',
       'RI', 'NJ', 'NH', 'VT', 'MD', 'VA', 'DC', 'PA', 'WV', 'DE', 'FL',
       'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'OH', 'KS', 'KY', 'LA', 'ME',
       'QC', 'NB', 'MI', 'MN', 'WI', 'MO', 'MS', 'MT', 'NC', 'SC', 'ND',
       'NE', 'NM', 'OK', 'OR', 'SD', 'TN', 'TX', 'WA', 'WY'], dtype=object)

In [22]:
dogs[dogs['sex']=='Unknown']['sex']

5780     Unknown
34991    Unknown
34992    Unknown
Name: sex, dtype: object

In [23]:
res = dogs.pivot_table(index='contact_state', columns='sex', aggfunc='size', fill_value=0)
res.drop('Unknown', axis=1, inplace=True)
res['imbalance'] = abs(res['Male'] - res['Female'])
res

sex,Female,Male,imbalance
contact_state,,,
AK,7,8,1
AL,716,712,4
AR,351,344,7
AZ,1067,1181,114
CA,777,887,110
CO,912,861,51
CT,682,740,58
DC,176,160,16
DE,148,148,0


In [24]:
massimo= res['imbalance'].max()
res[res['imbalance'] == massimo]

sex,Female,Male,imbalance
contact_state,,,
OH,1234,1439,205


# 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [25]:
dogs.groupby(['age' , 'size'])[['stay_duration' , 'stay_cost']].mean()

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

# 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [26]:
num_viaggi = travels.groupby(['id'], as_index=False).size()
viaggi3 = num_viaggi[num_viaggi['size'] >= 3]
pd.merge(dogs[['id', 'breed_primary' ]] , viaggi3 , on = 'id')


,id,breed_primary,size
0,45642530,Jindo,4
1,46039420,Border Collie,4
2,40036107,Pit Bull Terrier,3
3,45851842,Labrador Retriever,3
4,45841145,Mixed Breed,3
...,...,...,...
558,41144335,Chihuahua,3
559,40103682,Rat Terrier,3
560,38664932,Pit Bull Terrier,3
561,38495992,Pit Bull Terrier,3


# 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

In [27]:
travels['found'].mask(travels['manual'].notnull() , travels['manual'] , inplace=True)
travels[['found' , 'manual']]

,found,manual
0,Arkansas,NaN
1,Bahamas,Bahamas
2,Maryland,Maryland
3,Adaptil,NaN
4,Afghanistan,NaN
...,...,...
6189,WV,NaN
6190,Wyoming,NaN
6191,Yazmin,NaN
6192,Ohio,Ohio


# 9. For each state, compute the ratio between the number of travels and the population.

In [28]:
travels['contact_state'].unique()

array(['MN', 'FL', 'MD', 'CO', 'CT', 'OH', 'AL', 'NY', 'NJ', 'PA', 'VA',
       'GA', 'ME', 'NH', 'MI', 'VT', 'TN', 'WI', 'NM', 'OR', 'WA', 'IA',
       'KY', 'NV', 'UT', 'AZ', 'NC', 'AR', 'MA', 'RI', 'OK', 'CA', 'IN',
       'SC', 'IL', 'MO', 'TX', 'DC', 'KS', 'DE', 'WV', 'NB', 'MS', 'LA',
       '17325'], dtype=object)

In [29]:
travels[travels['contact_state'] == '17325']

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
3237,3237,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,New York,NaN,True,NaN
3238,3238,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",New York,NaN,True,NaN
3714,3714,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Pennsylvania,NaN,True,NaN
3715,3715,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Pennsylvania,NaN,True,NaN
6029,6029,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Virginia,NaN,True,NaN
6030,6030,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Virginia,NaN,True,NaN
6074,6074,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Washington DC,NaN,True,NaN
6075,6075,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Washington DC,NaN,True,NaN
6133,6133,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,West Virginia,NaN,True,NaN
6134,6134,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",West Virginia,NaN,True,NaN


In [30]:
travels['contact_state'].mask(travels['contact_state']=='17325' , travels['contact_city'] , inplace=True)
travels[travels['contact_city']=='PA']

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
2472,2472,36978896,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,Maryland,NaN,True,NaN
2473,2473,33218331,PA,PA,"Born in August 2014, Bucky has a great sense o...",Maryland,NaN,True,NaN
3190,3190,36978896,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,New Jersey,NaN,True,NaN
3191,3191,33218331,PA,PA,"Born in August 2014, Bucky has a great sense o...",New Jersey,NaN,True,NaN
3237,3237,36978896,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,New York,NaN,True,NaN
3238,3238,33218331,PA,PA,"Born in August 2014, Bucky has a great sense o...",New York,NaN,True,NaN
3714,3714,36978896,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,Pennsylvania,NaN,True,NaN
3715,3715,33218331,PA,PA,"Born in August 2014, Bucky has a great sense o...",Pennsylvania,NaN,True,NaN
6029,6029,36978896,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,Virginia,NaN,True,NaN
6030,6030,33218331,PA,PA,"Born in August 2014, Bucky has a great sense o...",Virginia,NaN,True,NaN


In [31]:
lista = pd.read_csv('stati.csv')
lista

,state,abbrev,code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA
5,Colorado,Colo.,CO
6,Connecticut,Conn.,CT
7,Delaware,Del.,DE
8,District of Columbia,D.C.,DC
9,Florida,Fla.,FL


In [32]:
statipop = pd.merge(states, lista, on='state')
statipop

,state,population,abbrev,code
0,Alabama,5.024.279,Ala.,AL
1,Alaska,733.391,Alaska,AK
2,Arizona,7.151.502,Ariz.,AZ
3,Arkansas,3.011.524,Ark.,AR
4,California,39.538.223,Calif.,CA
5,Colorado,5.773.714,Colo.,CO
6,Connecticut,3.605.944,Conn.,CT
7,Delaware,989.948,Del.,DE
8,District of Columbia,689.545,D.C.,DC
9,Florida,21.538.187,Fla.,FL


In [33]:
prova = travels.groupby('contact_state',as_index=False).size()
tra_sta = pd.merge(statipop, prova, left_on='code' , right_on='contact_state')
tra_sta['ratio'] = tra_sta['size'] / tra_sta['population'].str.replace('\.' , '',regex= True).astype(int)
tra_sta[['state', 'population' , 'size' , 'ratio']]

,state,population,size,ratio
0,Alabama,5.024.279,75,1.492751e-05
1,Arizona,7.151.502,70,9.788154e-06
2,Arkansas,3.011.524,10,3.320578e-06
3,California,39.538.223,28,7.081755e-07
4,Colorado,5.773.714,103,1.783947e-05
5,Connecticut,3.605.944,90,2.495879e-05
6,Delaware,989.948,57,5.757878e-05
7,District of Columbia,689.545,112,1.624259e-04
8,Florida,21.538.187,133,6.175079e-06
9,Georgia,10.711.908,109,1.017559e-05


# 10. For each dog, compute the number of days from the posted day to the day of last access.

In [34]:
dogs['posted'] = dogs['posted'].dt.tz_localize(None)
dogs['posted'] = dogs['posted'].dt.date

dogs['accessed'] = pd.to_datetime(dogs['accessed'] , format="%Y-%m-%d")
dogs['accessed'] = (dogs['accessed']).dt.date
 

In [37]:
dogs[['accessed','posted']] 

,accessed,posted
0,2019-09-20,2019-09-20
1,2019-09-20,2019-09-20
2,2019-09-20,2019-09-20
3,2019-09-20,2019-09-20
4,2019-09-20,2019-09-20
...,...,...
58175,2019-09-20,2019-05-03
58176,2019-09-20,2019-04-13
58177,2019-09-20,2018-09-27
58178,2019-09-20,2018-09-12


In [38]:
dogs['days_diff'] = (dogs['accessed'] - dogs['posted']).dt.days  
dogs[['id' , 'days_diff']]

,id,days_diff
0,46042150,0
1,46042002,0
2,46040898,0
3,46039877,0
4,46039306,0
...,...,...
58175,44605893,140
58176,44457061,160
58177,42865848,358
58178,42734734,373


# 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

In [39]:
dogs['weeks_diff'] = abs(dogs['days_diff'])//7
dogs.sort_values(by=['weeks_diff'])[['id' , 'weeks_diff']].set_index(['weeks_diff'])

,id
weeks_diff,
0,46042150
0,45968226
0,45968176
0,45968114
0,45968101
...,...
729,5142790
746,4527948
811,2613506


# 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

In [40]:
comp = dogs[dogs.duplicated(subset=['breed_primary' , 'sex'],keep=False)][['id','breed_primary', 'sex','description']]
comp['description'] = comp['description'].astype(str)
comp = comp[comp['description'] != 'nan']
comp.set_index(['breed_primary' , 'sex'], inplace=True)
comp

id  \
breed_primary                  sex                
American Staffordshire Terrier Male    46042150   
Pit Bull Terrier               Male    46042002   
Shepherd                       Male    46040898   
Dachshund                      Male    46039306   
Boxer                          Male    46039304   
...                                         ...   
Australian Shepherd            Male    45168741   
Pit Bull Terrier               Male    44843897   
                               Male    44658860   
Border Collie                  Male    44605893   
                               Female  42865848   

                                                                             description  
breed_primary                  sex                                                        
American Staffordshire Terrier Male    Harley is not sure how he wound up at shelter ...  
Pit Bull Terrier               Male    6 year old Biggie has lost his home and really...  
Shepherd                       Male    Approx 2 years old.\n Did I catch your eye? I ...  
Dachshund                      Male    Theo is a friendly dachshund mix who gets alon...  
Boxer                          Male    Oliver was born around mid-June and came to us...  
...                                                                                  ...  
Australian Shepherd            Male    Charlie was adopted from us 5 years ago, but r...  
Pit Bull Terrier               Male    Samson is a dog that will need someone to show...  
                               Male    Buddy was an owner surrender by an older gentl...  
Border Collie                  Male    Due to the small size of our volunteer base, w...  
                               Female  Due to the small size of our volunteer base, w...  

[49437 rows x 2 columns]

In [41]:
comp.sort_index(inplace=True)
comp

id  \
breed_primary     sex                
Affenpinscher     Female  45889013   
                  Female  22427951   
                  Female  45970614   
                  Female  45871731   
                  Female  45916762   
...                            ...   
Yorkshire Terrier Male    45958435   
                  Male    45922985   
                  Male    45466492   
                  Male    45985506   
                  Male    45945521   

                                                                description  
breed_primary     sex                                                        
Affenpinscher     Female  This cutie is very sweet. She is a little shy ...  
                  Female  To see even more click the following link:\n\n...  
                  Female  Elsie.jpgThank you for looking at our availabl...  
                  Female  55351.jpgThank you for looking at our availabl...  
                  Female  Norma Jean is an Affenpinscher mix girl that w...  
...                                                                     ...  
Yorkshire Terrier Male    For a quicker response please submit a online ...  
                  Male    You can fill out an adoption application onlin...  
                  Male    My name is Juno from California and I am a won...  
                  Male    2 years, 4 pounds, cute, lovable, friendly, aw...  
                  Male    Energetic 8 yo male.\nHe is selective about ot...  

[49437 rows x 2 columns]

In [42]:
def pulizia(x):
    string1_words = set(x['description'].split())
    unwanted_characters = ".,!?)(][:;-_ /*"
    string1_words = {word.strip(unwanted_characters).lower() for word in string1_words}
    return(string1_words)
    

In [43]:
comp['description'] = comp['description'].str.replace("\n" , " ")
comp['description'] = comp['description'].apply(lambda x: fix_encoding(x))
comp['words'] = comp.apply(pulizia, axis=1)

In [44]:
from collections import Counter
def similarity(words1 , words2):
    a = Counter(words1)
    b = Counter(words2)
    intersect=[]
    space = max(len(words1) , len(words2))
    for i in set(words1).intersection(set(words2)):
        intersect.extend([i] * min(a[i] , b[i]))
        
    return (len(intersect) / space)

In [45]:
def test(x: pd.DataFrame):    
    found = list()
    for index1, row1 in x.iloc[1:].iterrows():
        for index2, row2 in x.iloc[::-1].iterrows():
            if row1['id'] == row2['id']:
                break
            elif similarity(row1['words'] , row2['words']) >= 0.9:
                found.append([row1['id'], row2['id']])

    return found

In [46]:
coppie = comp.groupby(axis=0 , level=[0,1]).apply(test)

In [47]:
coppie

breed_primary                      sex   
Affenpinscher                      Female                               [[45970614, 45871731]]
                                   Male                                                     []
Afghan Hound                       Male                                                     []
Airedale Terrier                   Female                                                   []
                                   Male                                                     []
                                                                   ...                        
Xoloitzcuintli / Mexican Hairless  Male                                                     []
Yellow Labrador Retriever          Female    [[46023037, 46022945], [46023037, 46023018], [...
                                   Male           [[44266927, 44267041], [46014773, 46014715]]
Yorkshire Terrier                  Female    [[41166946, 41166968], [45251495, 42906237], [...
        

In [53]:
coppiedf=pd.DataFrame(coppie)
coppiedf['numero'] = coppiedf[0].apply(lambda x:len(x))
coppiedf['numero'].sum()

7736